# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

and the second audio can be download from this video link : https://www.youtube.com/watch?v=ZGgBuJE0-s4&ab_channel=CNET 

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!


In [11]:

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

print("Hello World")

True
NVIDIA A100-SXM4-40GB
Hello World


In [12]:
# !pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

In [13]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
# from google.colab import drive
from huggingface_hub import login
# from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [14]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [15]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

# drive.mount("/content/drive")
audio_filename = "denver_extract.mp3"

# Download denver_extract.mp3

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [17]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)

# Get the Hugging Face token from environment variables
hf_token = os.getenv('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [19]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)

# Get the OpenAI API key from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [39]:
# Use the Whisper OpenAI model to convert the Audio to Text


# the audio file is sent to OpenAI's servers, where the Whisper model runs in the cloud. The model itself is never downloaded to your machine or any directory. 
# Only your audio file is uploaded, and the transcribed text is returned.
audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

Good morning and welcome to Build. Here we are in 2025 building out this open agentic web at scale. We're going from these few apps with vertically integrated stacks to more of a platform that enables this open scalable agentic web. More importantly, it's all about expanding that opportunity for developers across every layer. We have a bunch of new updates. We are rolling out at Build starting with Visual Studio. It is the most powerful IDE for.NET and C++, and we're making it even better. .NET 10 support, live preview at design time, improvements to get tooling, a new debugger for cross-platform apps and much, much more. When it comes to VS Code, just a couple of weeks ago, we shipped our 100th release in the open. It included improved multi-window support and made it easier to view stage directly from within the editor. GitHub continues to be the home for developers. GitHub Enterprise has tremendous momentum in the enterprise, and we're doubling down for developers building any appli

In [21]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [22]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [23]:
import os
os.environ["HF_HOME"] = "/media/exxact1/llms/"

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings b

In [ ]:
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [ ]:
salLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [26]:
response = tokenizer.decode(outputs[0])

In [27]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism and you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America, and to the republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. All right. Thank you, Councilman Lopez. Madam Secretary, roll call. Clerk. Here. Espinosa. Here. Flynn. Here. Gilmour. Here. Here. Cashman. Here. Kenneche. Here. Lopez. Here. New. Here. Ortega. Here. Sussman. Here. Mr. President. Here. 11 present. 11 members present. We do have a quorum. Approval of the minutes. Seeing none. Minutes of October 2nd stand approved. Council announcements. Are there any announcements by members of Council? Councilman Clark. Thank you, Mr. President. I just wanted to invite everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st at 6 o'clock p.m. It will move along Broadway from 3rd to Alameda. It's going to be a fun, family-friendly event. Everyone's invited to come down, wear a costume. There will be candy for the kids and there are tiki zombies and 29 hearses and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at 6 o'clock for the Broadway Halloween parade. Thank you, Mr. President. All right. Thank you, Councilman Clark. I will be there. All right. Presentations. Madam Secretary, do we have any presentations? None, Mr. President. Communications. Do we have any communications? None, Mr. President. We do have one proclamation this evening. Proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver. Councilman Lopez, will you please read it? Thank you, Mr. President, with pride. Proclamation number 17, well, let me just say this differently. Proclamation number 1127, series of 2017, an observance of the second annual Indigenous Peoples Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the Indigenous Peoples have lived and flourished on the lands known as the Americas since time immemorial and that Denver and the surrounding communities are built upon the ancestral homelands of numerous Indigenous tribes, which include the Southern Ute, the Ute Mountain, Ute tribes of Colorado. And whereas the tribal homelands and seasonal encampments of the Arapaho and Cheyenne people along the banks of the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes and the City and County of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas on October 3rd, 2016, the City and County of Denver unanimously passed Council Bill 801, series of 2016, officially designating the second Monday of October of each year as Indigenous Peoples Day in Denver, Colorado. And whereas the Council of the City and County of Denver continues to recognize and value the vast contributions made to the community through Indigenous Peoples' knowledge, science, philosophy, arts, and culture. And through these contributions, the City of Denver has developed and thrived. Whereas the Indigenous community, especially youth, have made great efforts this year to draw attention to the contributions of Indigenous people, including Confluence Week, drawing record attendance to a National Indigenous Youth Leadership Conference, leading conversations on inclusion with their peers, and supporting increased Indigenous youth participation in science and engineering. Now, therefore, be it proclaimed by the Council of the City and County of Denver, Section 1, that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of Indigenous people to our history, our past, our present, and future, and continues to promote the education of the Denver community about these historical and contemporary contributions of Indigenous people. Section 2, that the City and County of Denver, Colorado, does hereby observe October 9th, 2017, as Indigenous Peoples Day. Section 3, that the Clerk of the City and County of Denver shall attest and affix the seal of the City and County of Denver to this proclamation, and that a copy be transmitted to the Denver American Indian Commission, the City and County of Denver School District No. 1, and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez. Your motion to adopt. Mr. President, I move that Proclamation No. 1127, Series of 2017, be adopted. All right. It has been moved and seconded. It comes from the members of Council. Councilman Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples Day in Denver, officially for the second time. It's always awesome to be able to see not just this proclamation come by my desk, but to see so many different people from our community in our Council Chambers. It was a very beautiful piece of artwork that you presented to us earlier, and it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous Peoples Day when we sat down and wrote it, and as a community, we couldn't think of anything else to begin except for the confluence of the two rivers, and those confluence of the two rivers created such a great city, and we live in such an amazing city, and we're all proud of it, and sometimes we, and a lot of people from all over the country or all over the world are proud of it, and sometimes a little too proud of it is telling them to go back home, but I'm kidding when I say that, but the really nice thing about this is that we are celebrating Indigenous Peoples Day out of pride for who we are, who we are as a city, and the contributions of Indigenous people to the city, not out of spite, not out of a replacement of one culture over the other, or out of contempt or disrespect. You know, I think of a quote that Cesar Chavez made very popular, and it stuck with me for a very long time, and any time I have the opportunity to speak in front of children, and especially children in our community that, you know, they often second guess themselves on where they're coming from, who they are, and I always say that, you know, it's very important to be proud of who you're from, and the quote that I use from Cesar Chavez is, you know, pride in one's own cultures does not require contempt or disrespect of another, right? And that's very important. It's very important for us to recognize that, no matter who we are, where we come from in this society, that your pride in your own culture doesn't require, should not require the contempt or disrespect of another. And man, what a year to be, for that to just sit on our shoulders for a while, for us to think about, right? And so, I wanted to, just to thank you all, to thank the commission, there's going to be a couple individuals that are going to come speak, thank you for your art, your lovely artwork, for us to see what's in your heart and what now has become, probably is going to be a very important symbol for the community. And also, just for the work, the daily work, every single day, we still have a lot of brothers and sisters whose ancestors once lived in these lands freely, now stand on street corners, right? In poverty, without access to services, right? Without access to sobriety or even housing or jobs. And what a, what a, what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So, we have a lot of work to do. And these kind of proclamations and this day is not a day off, it's a day on in Denver, right? And addressing those critical issues. So, I know that my colleagues are very supportive, I'm going to ask you to support this proclamation, as I know you always have done in the past, I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again, today, as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Ortega? Mr. President, I want to ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was created, and had the benefit of being able to go down to the Four Corners for a peace treaty signing ceremony between the Utes and the Comanches that had been sort of at odds with each other for about a hundred years, and just being able to participate in that powwow was pretty awesome. And for those of you who continue to participate in the annual powwow, it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on as well as the Native language from generation to generation is just so incredible because in so many cultures, you know, people have come here and assimilated to the, you know, the norms here and they lose their language and lose a lot of the culture. And in the Native community, that hasn't happened. That, you know, commitment to just passing that on from generation to generation is so important. And so I'm happy to be a co-sponsor of this tonight. Thank you. All right. Thank you, Councilwoman Ortega. Councilwoman Kaneech. Thank you very much, and I also want to thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is, you know, the words sometimes don't convey the power of imagery or music or the other pieces that make up culture, and so I think the art is so important. And when you talked about water, I was also thinking about land, and I guess I just wanted to say thank you. Many of the Native American peoples of Colorado have been at the forefront, or actually nationally, of defending some of the public lands that have been protected over the last few years that are under attack right now. And there are places that the communities have fought to protect but that everyone gets to enjoy. And so I just think that it's an example of where cultural preservation intersects with environmental protection, with, you know, recreation and all of the other ways that public lands are so important. And so I think I just wanted to say thank you for that because I think we have some very sacred places in our country that are at risk right now. And so as we celebrate, I appreciate that there's still a piece of resistance in here, and I think that I just want to mention a solidarity and mention a feeling of solidarity with that resistance. So thank you, and happy Confluence Week. Thank you, Councilwoman Knitsch. And seeing no other comments, I'll just say a couple. And in a time of such divisive ugliness and just despicable behavior from our leadership, the reason I'm so supportive of Indigenous Peoples' Day is because it means inclusivity. It means respecting all, respecting those who have been silenced on purpose for a long time and whose history has not been told. And so we celebrate inclusivity in the face of such evil times, honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Denver City Council Meeting**
==============================================

**Summary**
------------

The Denver City Council meeting was held on October 9th, 2017, with 11 members present. The meeting included the approval of the minutes from the previous meeting, Council announcements, presentations, communications, and a proclamation. The main topic of the meeting was the observance of Indigenous Peoples Day in the City and County of Denver.

**Attendees**
------------

* 11 members of the Denver City Council: Espinosa, Flynn, Gilmour, Cashman, Kenneche, Lopez, New, Ortega, Sussman, Clark, and Knitsch

**Location and Date**
----------------------

* Date: October 9th, 2017
* Location: Denver City Council Chambers

**Key Discussion Points**
---------------------------

* The approval of the minutes from the previous meeting
* Council announcements, including an invitation to the first-ever Halloween parade in the Lucky District 7
* Presentations, including a proclamation for the observance of Indigenous Peoples Day in the City and County of Denver
* Communications, including a proclamation for the observance of Indigenous Peoples Day in the City and County of Denver
* A presentation of the logo for Confluence Week, highlighting the importance of water and the confluence of two rivers

**Takeaways**
--------------

* The Denver City Council recognizes the importance of Indigenous Peoples Day and the contributions of Indigenous people to the city's history and culture.
* The Council celebrates the cultural and foundational contributions of Indigenous people to the city's history, past, present, and future.
* The Council promotes the education of the Denver community about the historical and contemporary contributions of Indigenous people.

**Action Items with Owners**
---------------------------

* **Proclamation No. 1127, Series of 2017**: Adopted by the Denver City Council, with Councilman Lopez as the main sponsor.
* **Confluence Week**: Celebrated in the City and County of Denver, with a focus on highlighting Indigenous events and sharing the idea of Indigenous Peoples Day.
* **Halloween Parade**: Organized by Councilman Clark, to be held on October 21st, 2017, in the Lucky District 7.

**Next Steps**
----------------

* The Clerk of the City and County of Denver shall attest and affix the seal of the City and County of Denver to the proclamation.
* A copy of the proclamation shall be transmitted to the Denver American Indian Commission, the City and County of Denver School District No. 1, and the Colorado Commission on Indian Affairs.<|eot_id|>

In [32]:
audio_filename = "Microsoft Build 2025 Keynote_ Everything Revealed, in 14 Minutes.mp3"
audio_file = open(audio_filename, "rb")
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

Good morning and welcome to Build. Here we are in 2025 building out this open agentic web at scale. We're going from these few apps with vertically integrated stacks to more of a platform that enables this open scalable agentic web. More importantly, it's all about expanding that opportunity for developers across every layer. We have a bunch of new updates. We are rolling out at Build starting with Visual Studio. It is the most powerful IDE for.NET and C++, and we're making it even better. .NET 10 support, live preview at design time, improvements to get tooling, a new debugger for cross-platform apps and much, much more. When it comes to VS Code, just a couple of weeks ago, we shipped our 100th release in the open. It included improved multi-window support and made it easier to view stage directly from within the editor. GitHub continues to be the home for developers. GitHub Enterprise has tremendous momentum in the enterprise, and we're doubling down for developers building any appli

In [33]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Microsoft Build Keynote  meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [34]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

In [43]:


inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
attention_mask = (inputs != tokenizer.pad_token_id).long()  # create attention mask

streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(
    inputs,
    attention_mask=attention_mask,
    max_new_tokens=2000,
    streamer=streamer,
    pad_token_id=tokenizer.pad_token_id
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Microsoft Build Keynote  meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
Good morning and welcome to Build. Here we are in 2025 building out this open agentic web at scale. We're going from these few apps with vertically integrated stacks to more of a platform that enables this open scalable agentic web. More importantly, it's all about expanding that opportunity for developers across every layer. We have a bunch of new updates. We are rolling out at Build starting with Visual Studio. It is the most powerful ID

In [41]:
response = tokenizer.decode(outputs[0])

In [42]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Microsoft Build Keynote  meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
Good morning and welcome to Build. Here we are in 2025 building out this open agentic web at scale. We're going from these few apps with vertically integrated stacks to more of a platform that enables this open scalable agentic web. More importantly, it's all about expanding that opportunity for developers across every layer. We have a bunch of new updates. We are rolling out at Build starting with Visual Studio. It is the most powerful IDE for.NET and C++, and we're making it even better..NET 10 support, live preview at design time, improvements to get tooling, a new debugger for cross-platform apps and much, much more. When it comes to VS Code, just a couple of weeks ago, we shipped our 100th release in the open. It included improved multi-window support and made it easier to view stage directly from within the editor. GitHub continues to be the home for developers. GitHub Enterprise has tremendous momentum in the enterprise, and we're doubling down for developers building any applications, trust, security, compliance, auditability, data residency are even more critical today. As GitHub Copilot has evolved inside VS Code, AI has become so central to how we code. That's why we're open-sourcing Copilot in VS Code. We're really excited about this. Starting today, we will integrate these AI-powered capabilities directly into the core of VS Code, bringing them into the same open-source repo that powers the world's most loved DevTool. In fact, we're building app modernization right into agent mode. So Copilot now is capable of upgrading frameworks like a Java 8 to Java 21 or.NET 6 to.NET 9, and migrate any on-premise app to the Cloud. The next thing we're introducing is an autonomous agent for Site Reliability Engineering or SRE. The SRE agent starts automatically triaging, root causing, mitigating the issue, and then it logs the incident management report as a GitHub issue with all the repair items. From there, you can even assign the repair items to GitHub Copilot, a full coding agent built right into GitHub, taking Copilot from being a pair programmer to a peer programmer. You can assign issues to Copilot, bug fixes, new features, code maintenance, and it'll complete these tasks autonomously. Today, I'm super excited that it's now available to all of you. I don't think since Teams launched, we've had an update of this level, and it really brings together chat, search, notebooks, create, and agents all into this one scaffolding that's intuitive. I always say this is the UI for AI. Chat, for example, is grounded both on the web data as well as your work data, and that's the game changer especially with pages. Search works across all of your applications, whether it's Confluence or Google Drive or Jira or ServiceNow, not just M365 data. With notebooks, I can now create these heterogeneous collections of data. In fact, I can have chats and pages and any documents, e-mails, all in that collection. Then, in fact, I can get all these audio reviews or podcasts out of it. I can use create to turn a PowerPoint into a new explainer video or generate an image. When it comes to agents, we have a couple of special agents like researcher. It's been perhaps the biggest game changer for me because it's synthesizing across both the web and enterprise sources, applying deep chain of thought reasoning to any topic or any project. Analyst goes from raw data across multiple source files. I can just upload a bunch of Excel files. It will get the insights. It'll do forecast. It'll do all the visualizations. All of the agents you build can now show up in Teams and in Copilot, and you can ask questions, assign action items, or kick off a workflow by just at mentioning an agent in a chat or meeting. With the Teams AI library, building multiplayer agents is easier than ever. It now supports MCP, and with just one line of code, you can even have it enable A2A. You can add things like episodic or semantic memory by using Azure Search and new retrieval system, which I'll talk about later. As a developer, you can now publish, and this is the biggest thing. Right now, you can build an agent. You can publish your agent to the agent store and have them discovered and distributed across both Copilot and Teams, providing you access to the hundreds of millions of users and unlocking that opportunity. Today, we are introducing a new class of enterprise-grade agents you can build using models fine-tuned on your company's data, workflows, and style. We call it Copilot Tuning. Copilot can now learn your company's unique tone, and language, and soon, it'll even go further understanding all of the company's specific expertise and knowledge. All you need to do is seed the training environment with a small set of references and kick off a training run. The customized model inherits the permissions of all the source control, and once integrated into the agent, it can deploy to authorized users. Our new model router will automatically choose the best OpenAI model for the job, no more manual model selections. An approach today, though, goes from having apps that you build or agents you build only bind to one model to truly becoming multi-model. That's why today, we're thrilled to announce Grok from XAI is coming to Azure. When you have multiple models, what you need is a new capability in how you use these models, and now you can provision throughput once on Foundry, and you can use that provision throughput across multiple models, including Grok. That's just a game changer in terms of how you think about models and model provisioning. The Foundry agent service lets you build declarative agents, in fact, just with few lines of code just in the portal. For complex workflows, it supports multi-agent orchestration. I'm excited to share that now the agent service is generally available. We're making it straightforward, for example, for you to connect Foundry to your container app or functions, and deploy any open-source model into AKS, whether it's in the Cloud or in hybrid mode with Arc. You can now take a model, fine-tune it or post-train it in Foundry, and then drop it right into Copilot Studio so that you can now use that post-train model to automate a workflow or build an agent. This healthcare agent orchestrator that Stanford used is now available to everyone in Foundry. It's pretty awesome. We now have new observability features coming to Foundry to help you monitor and manage AI in production. You can track the impact, quality, safety, as well as cost all in one place. With Entra ID, agents now get their own identity, permissions, policies, access controls. The agents you build in Foundry and Copilot Studio show up automatically in an agent directory in Entra. We're also partnering with ServiceNow and Workday to bring automated provisioning and management to their agents via Entra. When it comes to data governance, Purview now integrates with Foundry. When you write an agent automatically because of Purview, you can ensure end-to-end data protection, another massive safety consideration. On the security side, Defender now integrates with Foundry. That means your agents are also protected just like an endpoint would be from threats like wallet abuse or credential theft by Defender. Now, we want to bring the power of this app server and app building capability to the edge and clients as well with Foundry Local which we are announcing today. It includes a fast, high-performance runtime, models, agents as a service, and a CLI for local app development. Yes, it's fully supported on Windows and the Mac. We're excited to announce the Windows AI Foundry. Windows AI Foundry is what we used, in fact, ourselves internally to build features. Now, we're extending this platform to support the full dev life cycle, not just on co-pilot PCs, but across CPUs, GPUs, NPUs, all and in the Cloud. So you can build your application and have it run across all of that silicon. Foundry Local is built into Windows AI Foundry, so you can tap into this rich catalog of these pre-optimized open-source models that you can run locally on your device. We're announcing native support for MCP in Windows. Windows will now include several built-in MCP servers, like file systems, settings, app actions, as well as windowing. We're adding native MCP registry that lets MCP-compatible clients discover the secure MCP servers that have been vetted by us for security performance, all while keeping you in control. We first announced Bash on Ubuntu on Windows nearly 10 years ago. It subsequently became what we obviously call today WSL. Today, we are making WSL fully open-source. We're announcing today, and you all should go check out the code in the GitHub repo, NLWeb. It is a way for anyone who has a website or an API already to very easily make their website or their API an agentic application. We're integrating Cosmos DB directly into Foundry. So that means any agent can store and retrieve things like conversational history, and soon they'll be able to also use Cosmos for all their RAG application needs. We're taking it further with Azure Databricks, connecting your data in genie spaces or in AI functions to Foundry. The other very cool capability is now inside of a PostgreSQL query, you can have LLM responses directly integrated. We're bringing Cosmos DB to Fabric too, because AI apps need more than just structured data. They need semi-structured data, whether it's text, images, audio. With Cosmos and Fabric and your data instantly available alongside SQL, you can now unify your entire data estate and make it ready for AI. There's a lot more. In fact, we're even building our digital twin builder right into Fabric. Now, you can very easily take digital twins with no code, low code. As you can see here, you can map the data from your physical assets and systems super fast. We're also announcing shortcut transformations in one link. You can think of this as AI driven ETL. You can apply all these pre-built AI power transformations, audio to text or sentiment analysis when its data is coming in, summarization, all powered by Foundry straight into Fabric. In fact, the largest GB200-based supercomputer is going to be Azure. We're very, very excited about scaling this and making it available to all of you as developers. We're bringing together the entire stack I talked about today and saying, look, let's apply it to science and the scientific workflow, the scientific process. That's our ambition with Microsoft Discovery which we are announcing today. It understands the nuanced knowledge in the scientific domain from public domain as well as your own data if you're a biopharma company. Discovery is built on Foundry bringing advanced agents highly specialized in R&D, not just for reasoning but for conducting research itself. It's great to see how companies across every industry are already using Discovery to accelerate their R&D, and I can't wait to see this in the hands of more R&D labs all over and what they can do. That was a quick, comprehensive, whatever you want to call it, walk through the full stack and how we're creating new opportunity for you across the agentic web. We're taking really a systems approach, a platform approach which you can expect from Microsoft across every layer of the stack.<|eot_id|><|start_header_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of Microsoft Build Keynote Meeting**
==============================================

**Summary**
-----------

Date: 2025 (exact date not specified)
Location: Microsoft Build Conference
Attendees: Microsoft leadership and developers

**Key Discussion Points**
-------------------------

* Microsoft is expanding its platform to enable an open, agentic web at scale, providing opportunities for developers across every layer.
* New updates were announced for Visual Studio, including.NET 10 support, live preview at design time, and a new debugger for cross-platform apps.
* GitHub Copilot is being open-sourced and integrated into the core of VS Code, bringing AI-powered capabilities directly into the editor.
* Autonomous agents for Site Reliability Engineering (SRE) and GitHub Copilot were introduced, allowing for autonomous coding and issue assignment.
* Microsoft is introducing a new class of enterprise-grade agents, called Copilot Tuning, which can learn a company's unique tone, language, and expertise.
* The Foundry agent service is now generally available, allowing for declarative agents and multi-agent orchestration.
* Microsoft is announcing native support for MCP in Windows, as well as the Windows AI Foundry, which provides a full dev life cycle across CPUs, GPUs, NPUs, and in the Cloud.
* WSL is now fully open-source, and NLWeb is a new way for websites and APIs to become agentic applications.
* Cosmos DB and Azure Databricks are being integrated into Foundry, allowing for conversational history and RAG application needs.
* Digital twin builder is being built into Fabric, and shortcut transformations are being introduced in one link, allowing for AI-driven ETL.

**Takeaways**
------------

* Microsoft is expanding its platform to enable an open, agentic web at scale.
* New updates were announced for Visual Studio, GitHub Copilot, and the Foundry agent service.
* Autonomous agents and Copilot Tuning are being introduced to provide enterprise-grade AI capabilities.
* Microsoft is investing in native support for MCP in Windows and the Windows AI Foundry.

**Action Items**
---------------

* **Owner:** Microsoft leadership and developers
* **Action:** Implement new updates for Visual Studio, GitHub Copilot, and the Foundry agent service.
* **Deadline:** Not specified
* **Owner:** Microsoft leadership and developers
* **Action:** Develop and deploy autonomous agents and Copilot Tuning.
* **Deadline:** Not specified
* **Owner:** Microsoft leadership and developers
* **Action:** Provide native support for MCP in Windows and the Windows AI Foundry.
* **Deadline:** Not specified<|eot_id|>